## POS

In [1]:
from corpuscula.corpus_utils import download_ud, UniversalDependencies, \
                                    AdjustedForSpeech, syntagrus
import junky
from mordl import UposTagger

BERT_MODEL_FN = 'bert-base-multilingual-cased'
MODEL_FN = 'upos_model'
SEED=42

# we use UD Taiga corpus only as example. For real model training comment
# Taiga and uncomment SynTagRus
#corpus_name = 'UD_Russian-Taiga'
#corpus_name = 'UD_Russian-SynTagRus'
# download_ud(corpus_name, overwrite=False)
# corpus = UniversalDependencies(corpus_name)
corpus = syntagrus
#corpus = AdjustedForSpeech(corpus)

In [2]:
tagger = UposTagger()
tagger.load_train_corpus(corpus.train)
tagger.load_test_corpus(corpus.dev)

_ = tagger.train(MODEL_FN, device='cuda:10',
                 word_emb_type='bert', word_emb_model_device='cuda:9',
#                  word_emb_path='upos_' + BERT_MODEL_FN
#                                        + '_len512_ep3_bat8_seed42',
#                  word_emb_tune_params=None,
                 word_emb_path=None,
                 word_emb_tune_params={
                     'model_name': BERT_MODEL_FN,
                     'max_len': 512, 'epochs': 3, 'batch_size': 8
                 },
                 rnn_emb_dim=None, cnn_emb_dim=None, cnn_kernels=range(1, 7),
                 seed=SEED)

Train: Load corpus
[=================================================] 48814           
Corpus has been loaded: 48814 sentences, 871526 tokens
Test: Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


=== UPOS TAGGER TRAINING PIPELINE ===

BERT MODEL TUNING 'bert-base-multilingual-cased'. The result's model name will be 'upos_bert-base-multilingual-cased_len512_ep3_bat8_seed42'
Loading tokenizer...
Tokenizer is loaded. Vocab size: 119547
Corpora processing... done.
Loading model 'bert-base-multilingual-cased'... done.
Epoch 1: 100%|██████████| 48814/48814 [12:34<00:00, 64.67it/s, train_loss=0.0226] 
Average train loss: 0.08670417480997196
Dev: accuracy = 0.98225193
Dev: precision = 0.96434810
Dev: recall = 0.88591372
Dev: f1_score = 0.89677961
NB: Scores may be high because of tags stretching
Saving model to upos_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 2: 100%|██████████| 48814/48814 [12:26<00:00, 65.40it/s, train_loss=0.00107] 
Average train loss: 0.03311146485173351
Dev: accuracy = 0.98572055
Dev: precision = 0.96505439
Dev: recall = 0.93086512
Dev: f1_score = 0.94336020
NB: Scores may be high because of tags stretching
Saving model to upos_bert-base-multilingual

In [3]:
tagger = UposTagger()
tagger.load(MODEL_FN)

Loading dataset... done.
Creating model... done.
Loading state_dict... done.


In [4]:
junky.clear_tqdm()

In [5]:
_ = tagger.predict(corpus.dev, batch_size=256,
                   save_to='corpora/_dev_upos.conllu')

Load corpus
[===> 3800                                                         

Processing corpus
  0%|          | 0/6584 [00:00<?, ?it/s]

[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


100%|██████████| 6584/6584 [00:54<00:00, 120.35it/s]


In [6]:
_ = tagger.predict(corpus.dev, batch_size=128, clone_ds=True,
                   save_to='corpora/_dev_upos_.conllu')

Load corpus
[====> 4300                                                        

Vectorizing
100%|██████████| 52/52 [00:10<00:00,  4.74it/s]
Adjusting
100%|██████████| 6584/6584 [00:24<00:00, 268.73it/s]


[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [7]:
_ = tagger.evaluate(corpus.dev)

Evaluating UPOS
Processing corpus
  0%|          | 0/6584 [00:00<?, ?it/s]

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


100%|██████████| 6584/6584 [01:07<00:00, 98.19it/s] 
UPOS total: 118488
   correct: 117273
     wrong: 1215
  Accuracy: 0.9897457970427385


In [8]:
_ = tagger.evaluate(corpus.dev, 'corpora/_dev_upos.conllu')

Evaluating UPOS
Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
UPOS total: 118488
   correct: 117273
     wrong: 1215
  Accuracy: 0.9897457970427385


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [9]:
_ = tagger.evaluate(corpus.test)

Evaluating UPOS


Load corpus
[=====> 5900                                                       

Processing corpus
100%|██████████| 6491/6491 [01:05<00:00, 99.10it/s] 
UPOS total: 117329
   correct: 116332
     wrong: 997
  Accuracy: 0.9915025270819661


[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


In [10]:
corp = list(syntagrus.dev())
corp_test = list(tagger._get_corpus('corpora/_dev_upos.conllu'))
tags = sorted(set(x['UPOS'] for x in corp for x in x[0]
                                if x['UPOS']))
for tag in tags:
    print('{}: {}'.format(
        tag, tagger.evaluate(corp, corp_test,
                             label=tag, log_file=None)
    ))

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
ADJ: 0.9670146504079306
ADP: 0.9983960751014247
ADV: 0.9575892857142857
AUX: 0.9154315605928509
CCONJ: 0.9831574219627217
DET: 0.9402327776030198
INTJ: 0.3333333333333333
NOUN: 0.9895133482634816
NUM: 0.9574241617881852
PART: 0.958890290037831
PRON: 0.9642732049036777
PROPN: 0.95576199955762
PUNCT: 0.9994272623138603
SCONJ: 0.9386266094420601
SYM: 0.9259259259259259
VERB: 0.9795239894219872
X: 0.7758620689655172


## NER

In [11]:
import junky
from mordl import NeTagger

BERT_MODEL_FN = 'bert-base-multilingual-cased'
MODEL_FN = 'ne_model'
SEED=42

train_corpus = 'corpora/ner_train_upos.conllu'
dev_corpus = 'corpora/ner_dev_upos.conllu'
test_corpus = 'corpora/ner_test_upos.conllu'

In [12]:
tagger = NeTagger()
tagger.load_train_corpus(train_corpus)
tagger.load_test_corpus(dev_corpus)

_ = tagger.train(MODEL_FN, device='cuda:10',
                 word_emb_type='bert', word_emb_model_device='cuda:7',
#                  word_emb_path='misc-ne_' + BERT_MODEL_FN
#                                           + '_len512_ep3_bat8_seed42',
#                  word_emb_tune_params=None,
                 word_emb_path=None,
                 word_emb_tune_params={
                     'model_name': BERT_MODEL_FN,
                     'max_len': 512, 'epochs': 3, 'batch_size': 8
                 },
                 rnn_emb_dim=None, cnn_emb_dim=None, cnn_kernels=range(1, 7),
                 upos_emb_dim=60, seed=SEED)

Load corpus
[> 700                                                            

Train: 

[===============================] 30390                             
Corpus has been loaded: 30390 sentences, 378829 tokens
Load corpus
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
=== MISC:NE TAGGER TRAINING PIPELINE ===


Test: 


BERT MODEL TUNING 'bert-base-multilingual-cased'. The result's model name will be 'misc-ne_bert-base-multilingual-cased_len512_ep3_bat8_seed42'
Loading tokenizer...
Tokenizer is loaded. Vocab size: 119547
Corpora processing... done.
Loading model 'bert-base-multilingual-cased'... done.
Epoch 1: 100%|██████████| 30390/30390 [07:15<00:00, 69.86it/s, train_loss=0.506] 
Average train loss: 0.22115723773073204


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Dev: accuracy = 0.94016071
Dev: precision = 0.64285216
Dev: recall = 0.60400200
Dev: f1_score = 0.60815842
NB: Scores may be high because of tags stretching
Saving model to misc-ne_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 2: 100%|██████████| 30390/30390 [07:14<00:00, 69.94it/s, train_loss=0.113]  
Average train loss: 0.13452649930620503
Dev: accuracy = 0.94571112
Dev: precision = 0.67449232
Dev: recall = 0.65385552
Dev: f1_score = 0.65974808
NB: Scores may be high because of tags stretching
Saving model to misc-ne_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 3: 100%|██████████| 30390/30390 [07:14<00:00, 70.00it/s, train_loss=0.119]  
Average train loss: 0.0896721976198414
Dev: accuracy = 0.94637318
Dev: precision = 0.71431601
Dev: recall = 0.74003630
Dev: f1_score = 0.71776732
NB: Scores may be high because of tags stretching
Saving model to misc-ne_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%|██████████| 30390/303

In [13]:
tagger = NeTagger()
tagger.load(MODEL_FN)

Loading dataset... done.
Creating model... done.
Loading state_dict... done.


In [14]:
junky.clear_tqdm()

In [15]:
_ = tagger.predict(dev_corpus, save_to='corpora/_dev_ner.conllu')

Load corpus
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
Processing corpus
100%|██████████| 3799/3799 [00:28<00:00, 134.78it/s]


In [16]:
_ = tagger.predict(test_corpus, save_to='corpora/_test_ner.conllu')

Load corpus
[====] 3798                                                        
Corpus has been loaded: 3798 sentences, 47126 tokens
Processing corpus
100%|██████████| 3798/3798 [00:27<00:00, 135.96it/s]


In [17]:
_ = tagger.evaluate(dev_corpus)

Evaluating MISC:NE
Load corpus
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
Processing corpus
100%|██████████| 3799/3799 [00:28<00:00, 135.29it/s]
MISC:NE total: 17031
      correct: 14331
        wrong: 2700 [933 excess / 1017 absent / 750 wrong type]
     Accuracy: 0.841465562797252
[Total accuracy: 0.9428934010152284]


In [18]:
_ = tagger.evaluate(dev_corpus, 'corpora/_dev_ner.conllu', label=None)

Evaluating MISC:NE
Load corpus
Load corpus
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
MISC:NE total: 17031
      correct: 14331
        wrong: 2700 [933 excess / 1017 absent / 750 wrong type]
     Accuracy: 0.841465562797252
[Total accuracy: 0.9428934010152284]


In [19]:
_ = tagger.evaluate(test_corpus, clone_ds=True)

Evaluating MISC:NE
Load corpus
[====] 3798                                                        
Corpus has been loaded: 3798 sentences, 47126 tokens
MISC:NE total: 16718
      correct: 14064
        wrong: 2654 [1079 excess / 946 absent / 629 wrong type]
     Accuracy: 0.8412489532240699
[Total accuracy: 0.9436828926707126]


In [20]:
_ = tagger.evaluate(test_corpus, 'corpora/_test_ner.conllu')

Evaluating MISC:NE
Load corpus
Load corpus
[====] 3798                                                        
Corpus has been loaded: 3798 sentences, 47126 tokens
[====] 3798                                                        
Corpus has been loaded: 3798 sentences, 47126 tokens
MISC:NE total: 16718
      correct: 14064
        wrong: 2654 [1079 excess / 946 absent / 629 wrong type]
     Accuracy: 0.8412489532240699
[Total accuracy: 0.9436828926707126]


In [21]:
corp = list(tagger._get_corpus(dev_corpus, asis=True))
corp_test = list(tagger._get_corpus('corpora/_dev_ner.conllu'))
tags = sorted(set(x['MISC'].get('NE')
                      for x in corp for x in x[0]
                          if x['MISC'].get('NE')))
for tag in tags:
    print('{}: {}'.format(
        tag, tagger.evaluate(corp, corp_test,
                             label=tag, log_file=None)
    ))

Load corpus
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
Load corpus
[====] 3799                                                        
Corpus has been loaded: 3799 sentences, 47280 tokens
Address: 0.8393686165273909
City: 0.0
Date: 0.25
Department: 0.7639627659574468
Facility: 0.34297520661157027
Geo: 0.8395234758234057
Goal: 0.43557422969187676
Location: 0.46
Organization: 0.8485877076009213
Person: 0.7677902621722846
PersonProperty: 0.7095238095238096
Phone: 0.6839622641509434
Time: 0.4117647058823529


## FEATS

In [22]:
from corpuscula.corpus_utils import syntagrus
import junky
from mordl import FeatsTagger

BERT_MODEL_FN = 'bert-base-multilingual-cased'
MODEL_FN = 'featsj_model'
SEED=42

corpus = syntagrus

In [23]:
tagger = FeatsTagger()
tagger.load_train_corpus(corpus.train)
tagger.load_test_corpus(corpus.dev)

_ = tagger.train(MODEL_FN, device='cuda:9',
                 word_emb_type='bert', word_emb_model_device='cuda:8',
#                  word_emb_path='featsj_' + BERT_MODEL_FN
#                                          + '_len512_ep3_bat8_seed42',
#                  word_emb_tune_params=None,
                 word_emb_path=None,
                 word_emb_tune_params={
                     'model_name': BERT_MODEL_FN,
                     'max_len': 512, 'epochs': 3, 'batch_size': 8
                 },
                 rnn_emb_dim=None, cnn_emb_dim=None, cnn_kernels=range(1, 7),
                 upos_emb_dim=60, seed=SEED)

Train: Load corpus
[=================================================] 48814           
Corpus has been loaded: 48814 sentences, 871526 tokens
Test: Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


=== FEATSj TAGGER TRAINING PIPELINE ===

BERT MODEL TUNING 'bert-base-multilingual-cased'. The result's model name will be 'feats_bert-base-multilingual-cased_len512_ep3_bat8_seed42'
Loading tokenizer...
Tokenizer is loaded. Vocab size: 119547
Corpora processing... done.
Loading model 'bert-base-multilingual-cased'... done.
Epoch 1: 100%|██████████| 48814/48814 [12:36<00:00, 64.51it/s, train_loss=0.218] 
Average train loss: 0.4850327154421237


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Dev: accuracy = 0.94745143
Dev: precision = 0.62704412
Dev: recall = 0.64657384
Dev: f1_score = 0.62096613
NB: Scores may be high because of tags stretching
Saving model to feats_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 2: 100%|██████████| 48814/48814 [12:37<00:00, 64.42it/s, train_loss=0.115]  
Average train loss: 0.13618717225657984
Dev: accuracy = 0.95986793
Dev: precision = 0.72675114
Dev: recall = 0.74011768
Dev: f1_score = 0.72336798
NB: Scores may be high because of tags stretching
Saving model to feats_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 3: 100%|██████████| 48814/48814 [12:35<00:00, 64.65it/s, train_loss=0.0364] 
Average train loss: 0.07583380635661875
Dev: accuracy = 0.96507561
Dev: precision = 0.76576322
Dev: recall = 0.77419963
Dev: f1_score = 0.76197853
NB: Scores may be high because of tags stretching
Saving model to feats_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%|██████████| 48814/48814 [0

In [24]:
tagger = FeatsTagger()
tagger.load(MODEL_FN)

Loading dataset... done.
Creating model... done.
Loading state_dict... done.


In [25]:
junky.clear_tqdm()

In [26]:
_ = tagger.predict(corpus.dev, batch_size=256,
                   save_to='corpora/_dev_feats.conllu')

Load corpus
[======> 6500                                                      

Processing corpus
  0%|          | 0/6584 [00:00<?, ?it/s]

[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


100%|██████████| 6584/6584 [00:54<00:00, 119.98it/s]


In [27]:
_ = tagger.predict(corpus.dev, batch_size=128, clone_ds=True,
                   save_to='corpora/_dev_feats_.conllu')

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Vectorizing
100%|██████████| 52/52 [00:10<00:00,  4.75it/s]
Adjusting
100%|██████████| 6584/6584 [00:26<00:00, 253.22it/s]


In [28]:
_ = tagger.evaluate(corpus.dev, clone_ds=True)

Evaluating FEATS


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.58it/s]
Adjusting
100%|██████████| 6584/6584 [00:25<00:00, 260.45it/s]
FEATS total: 72911 tokens, 287256 tags
    correct: 70839 tokens, 283993 tags
      wrong: 2072 tokens, 3263 tags [495 excess / 532 absent / 2236 wrong type]
   Accuracy: 0.9715817914992251 / 0.988640794274097


In [29]:
_ = tagger.evaluate(corpus.dev, 'corpora/_dev_feats_.conllu')

Evaluating FEATS
Load corpus
[=> 1300                                                           

Load corpus
[=> 1300                                                           

[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
FEATS total: 72911 tokens, 287256 tags
    correct: 70839 tokens, 283993 tags
      wrong: 2072 tokens, 3263 tags [495 excess / 532 absent / 2236 wrong type]
   Accuracy: 0.9715817914992251 / 0.988640794274097


[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [30]:
_ = tagger.evaluate(corpus.test)

Evaluating FEATS


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Processing corpus
100%|██████████| 6491/6491 [01:05<00:00, 98.70it/s] 
FEATS total: 72281 tokens, 285208 tags
    correct: 70230 tokens, 281887 tags
      wrong: 2051 tokens, 3321 tags [508 excess / 640 absent / 2173 wrong type]
   Accuracy: 0.9716246316459374 / 0.9883558665956074


In [31]:
from corpuscula.corpus_utils import syntagrus
from morra import MorphParser

_ = MorphParser().evaluate_feats(gold=syntagrus.dev(),
                                 test='corpora/_dev_feats.conllu')

Evaluate FEATS
Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
[=======] 6584                                                     
FEATS total: 72911 tokens, 287256 tags
    correct: 70839 tokens, 283993 tags
      wrong: 2072 tokens, 3263 tags
   Accuracy: 0.9715817914992251 / 0.988640794274097


In [32]:
from morra.base_parser import _evaluate

_ = _evaluate(syntagrus.dev(), 'corpora/_dev_feats.conllu', what='feats')

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
Evaluate
[=======] 6584                                                     
   total: 72911 tokens, 287256 tags
 correct: 70839 tokens, 283993 tags
   wrong: 2072 tokens, 3263 tags
Accuracy: 0.9715817914992251 / 0.988640794274097


In [33]:
corp = list(syntagrus.dev())
corp_test = list(tagger._get_corpus('corpora/_dev_feats.conllu'))
tags = sorted(set(x for x in corp for x in x[0] for x in x['FEATS'].keys()))
for tag in tags:
    print('{}: {}'.format(
        tag, tagger.evaluate(corp, corp_test,
                             feat=tag, log_file=None)
    ))

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
Animacy: 0.9832340868358674
Aspect: 0.9910192444761226
Case: 0.9804652407439265
Degree: 0.9966783116659985
Foreign: 0.8723404255319149
Gender: 0.9856392928256803
Mood: 0.9970548017250447
Number: 0.9939010431995814
Person: 0.9977767896843042
Polarity: 0.7076923076923077
Tense: 0.994922984252646
Variant: 0.9909851899549259
VerbForm: 0.9978617248752673
Voice: 0.9878127004490058


## FEATS-sep

In [34]:
from corpuscula.corpus_utils import syntagrus
import junky
from mordl.feats_tagger import FeatsSeparateTagger

BERT_MODEL_FN = 'bert-base-multilingual-cased'
MODEL_FN = 'feats_model'
SEED=42

corpus = syntagrus

In [35]:
tagger = FeatsTagger()
tagger.load_train_corpus(train_corpus)
tagger.load_test_corpus(dev_corpus)
#tagger.remove_rare_feats(rel_thresh=.1)

_ = tagger.train(MODEL_FN, feats=None, device='cuda:10',
                 word_emb_type='bert',
#                  word_emb_path_suffix=BERT_MODEL_FN + '_len512_ep3_bat8_seed42',
#                  word_emb_model_device='cuda:7', word_emb_tune_params=None,
                 word_emb_path_suffix=None, word_emb_model_device='cuda:7',
                 word_emb_tune_params={
                     'model_name': BERT_MODEL_FN,
                     'max_len': 512, 'epochs': 3, 'batch_size': 8
                 },
                 rnn_emb_dim=None, cnn_emb_dim=None, cnn_kernels=range(1, 7),
                 upos_emb_dim=60, seed=SEED)

Train: Load corpus
[=================================================] 48814           
Corpus has been loaded: 48814 sentences, 871526 tokens
Test: Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


###### FEATS TAGGER TRAINING PIPELINE ######

We're gonna train separate models for all FEATS in train corpus. Feats are:

Animacy, Aspect, Case, Degree, Foreign, Gender, Mood, Number, Person, Polarity, Tense, Variant, VerbForm, Voice

=== FEATS:Animacy TAGGER TRAINING PIPELINE ===

BERT MODEL TUNING 'bert-base-multilingual-cased'. The result's model name will be 'feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42'
Loading tokenizer...
Tokenizer is loaded. Vocab size: 119547
Corpora processing... done.
Loading model 'bert-base-multilingual-cased'... done.
Epoch 1: 100%|██████████| 48814/48814 [12:29<00:00, 65.16it/s, train_loss=0.135]   
Average train loss: 0.06171073812945936


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Dev: accuracy = 0.98730239
Dev: precision = 0.73395393
Dev: recall = 0.72911685
Dev: f1_score = 0.73149664
NB: Scores may be high because of tags stretching
Saving model to feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 2: 100%|██████████| 48814/48814 [12:21<00:00, 58.09it/s, train_loss=0.00175] 
Average train loss: 0.024529414104471663
Dev: accuracy = 0.98909387
Dev: precision = 0.98026741
Dev: recall = 0.97831461
Dev: f1_score = 0.97928765
NB: Scores may be high because of tags stretching
Saving model to feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 3: 100%|██████████| 48814/48814 [12:14<00:00, 66.48it/s, train_loss=0.000714]
Average train loss: 0.01058906736019653
Dev: accuracy = 0.98940833
Dev: precision = 0.98085384
Dev: recall = 0.97852986
Dev: f1_score = 0.97968497
NB: Scores may be high because of tags stretching
Saving model to feats-animacy_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%

Epoch 25: 100%|██████████| 48814/48814 [02:55<00:00, 278.29it/s, train_loss=0.000569]
Epoch 25: 
Losses: train = 0.00050876, test = 0.01193807
Test: accuracy = 0.99474208
Test: precision = 0.98635770
Test: recall = 0.98425934
Test: f1_score = 0.98530454
new maximum score 0.99474208
Saving state_dict... done.
Epoch 26: 100%|██████████| 48814/48814 [02:54<00:00, 269.66it/s, train_loss=0.000542]
Epoch 26: 
Losses: train = 0.00050756, test = 0.01171744
Test: accuracy = 0.99442982
Test: precision = 0.98434117
Test: recall = 0.98462459
Test: f1_score = 0.98448280
BAD EPOCHS: 1 (<< <)
Epoch 27: 100%|██████████| 48814/48814 [02:55<00:00, 277.73it/s, train_loss=0.000514]
Epoch 27: 
Losses: train = 0.00051043, test = 0.01289623
Test: accuracy = 0.99441294
Test: precision = 0.98653115
Test: recall = 0.98278848
Test: f1_score = 0.98464714
BAD EPOCHS: 2 (<< <)
Epoch 28: 100%|██████████| 48814/48814 [02:55<00:00, 278.59it/s, train_loss=0.00051] 
Epoch 28: 
Losses: train = 0.00046409, test = 0.012461

...

new maximum score 0.99983965
Saving state_dict... done.
Epoch 3: 100%|██████████| 48814/48814 [02:55<00:00, 278.14it/s, train_loss=0.000218]
Epoch 3: 
Losses: train = 0.00023769, test = 0.00029232
Test: accuracy = 0.99983121
Test: precision = 0.97135259
Test: recall = 0.82352097
Test: f1_score = 0.88367872
BAD EPOCHS: 1 (<< <)
Epoch 4: 100%|██████████| 48814/48814 [02:55<00:00, 277.47it/s, train_loss=0.00018] 
Epoch 4: 
Losses: train = 0.00022394, test = 0.00031525
Test: accuracy = 0.99983121
Test: precision = 0.97135259
Test: recall = 0.82352097
Test: f1_score = 0.88367872
BAD EPOCHS: 2 (<< =)
Epoch 5: 100%|██████████| 48814/48814 [02:56<00:00, 276.84it/s, train_loss=0.000226]
Epoch 5: 
Losses: train = 0.00020455, test = 0.00022527
Test: accuracy = 0.99985653
Test: precision = 0.97362089
Test: recall = 0.85293273
Test: f1_score = 0.90445850
new maximum score 0.99985653
Saving state_dict... done.
Epoch 6: 100%|██████████| 48814/48814 [02:56<00:00, 277.24it/s, train_loss=0.000237]
Epoch

Saving state_dict... done.
Epoch 8: 100%|██████████| 48814/48814 [02:55<00:00, 291.69it/s, train_loss=0.000128]
Epoch 8: 
Losses: train = 0.00014044, test = 0.00160716
Test: accuracy = 0.99940078
Test: precision = 0.99016970
Test: recall = 0.98828497
Test: f1_score = 0.98921905
new maximum score 0.99940078
Saving state_dict... done.
Epoch 9: 100%|██████████| 48814/48814 [02:55<00:00, 277.37it/s, train_loss=0.000165]
Epoch 9: 
Losses: train = 0.00014490, test = 0.00204773
Test: accuracy = 0.99918979
Test: precision = 0.99089161
Test: recall = 0.98349496
Test: f1_score = 0.98709542
BAD EPOCHS: 1 (<< <)
Epoch 10: 100%|██████████| 48814/48814 [02:55<00:00, 288.40it/s, train_loss=0.000109]
Epoch 10: 
Losses: train = 0.00010617, test = 0.00179437
Test: accuracy = 0.99941766
Test: precision = 0.99048086
Test: recall = 0.98867778
Test: f1_score = 0.98957429
new maximum score 0.99941766
Saving state_dict... done.
Epoch 11: 100%|██████████| 48814/48814 [02:55<00:00, 278.69it/s, train_loss=7.54e-

Epoch 2: 100%|██████████| 48814/48814 [11:52<00:00, 68.53it/s, train_loss=0.00532] 
Average train loss: 0.0034185991946459986
Dev: accuracy = 0.99881838
Dev: precision = 0.98590701
Dev: recall = 0.98311415
Dev: f1_score = 0.98450638
NB: Scores may be high because of tags stretching
Saving model to feats-variant_bert-base-multilingual-cased_len512_ep3_bat8_seed42
Epoch 3: 100%|██████████| 48814/48814 [11:51<00:00, 68.61it/s, train_loss=1.69e-5] 
Average train loss: 0.001165160718217094
Dev: accuracy = 0.99894226
Dev: precision = 0.98730731
Dev: recall = 0.98497327
Dev: f1_score = 0.98613737
NB: Scores may be high because of tags stretching
Saving model to feats-variant_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%|██████████| 48814/48814 [00:32<00:00, 1521.70it/s]
Vectorizing
100%|██████████| 763/763 [01:16<00:00, 10.03it/s]
Adjusting
100%|██████████| 48814/48814 [02:50<00:00, 286.69it/s]
Tokenizing
100%|██████████| 6584/6584 [00:04<00:00, 1561.4

Epoch 3: 100%|██████████| 48814/48814 [11:53<00:00, 68.37it/s, train_loss=4.82e-5] 
Average train loss: 0.004595644683947441
Dev: accuracy = 0.99659332
Dev: precision = 0.98605947
Dev: recall = 0.98987735
Dev: f1_score = 0.98796037
NB: Scores may be high because of tags stretching
Saving model to feats-verbform_bert-base-multilingual-cased_len512_ep3_bat8_seed42

DATASETS CREATION
Tokenizing
100%|██████████| 48814/48814 [00:32<00:00, 1502.76it/s]
Vectorizing
100%|██████████| 763/763 [01:16<00:00, 10.03it/s]
Adjusting
100%|██████████| 48814/48814 [02:49<00:00, 287.16it/s]
Tokenizing
100%|██████████| 6584/6584 [00:04<00:00, 1492.31it/s]
Vectorizing
100%|██████████| 103/103 [00:10<00:00,  9.70it/s]
Adjusting
100%|██████████| 6584/6584 [00:22<00:00, 287.11it/s]

MODEL CREATION
Config saved

MODEL TRAINING
Epoch 1: 100%|██████████| 48814/48814 [02:55<00:00, 278.40it/s, train_loss=0.000348]
Epoch 1: 
Losses: train = 0.00343089, test = 0.00228097
Test: accuracy = 0.99887752
Test: precision = 

Epoch 19: 100%|██████████| 48814/48814 [02:55<00:00, 277.90it/s, train_loss=0.000238]
Epoch 19: 
Losses: train = 0.00027672, test = 0.00254399
Test: accuracy = 0.99868341
Test: precision = 0.98304285
Test: recall = 0.98030824
Test: f1_score = 0.98163145
BAD EPOCHS: 2 (<< >)
Epoch 20: 100%|██████████| 48814/48814 [02:55<00:00, 277.75it/s, train_loss=0.000245]
Epoch 20: 
Losses: train = 0.00027272, test = 0.00279070
Test: accuracy = 0.99860745
Test: precision = 0.98266673
Test: recall = 0.97983711
Test: f1_score = 0.98122879
BAD EPOCHS: 3 (<< <)
Epoch 21: 100%|██████████| 48814/48814 [02:57<00:00, 307.12it/s, train_loss=0.00032] 
Epoch 21: 
Losses: train = 0.00027421, test = 0.00251059
Test: accuracy = 0.99862433
Test: precision = 0.98281860
Test: recall = 0.98028608
Test: f1_score = 0.98152100
BAD EPOCHS: 3 (<< >)
Epoch 22: 100%|██████████| 48814/48814 [02:59<00:00, 285.99it/s, train_loss=0.000287]
Epoch 22: 
Losses: train = 0.00025931, test = 0.00280131
Test: accuracy = 0.99865809
Test

In [36]:
tagger = FeatsSeparateTagger()
tagger.load(MODEL_FN)

### Loading FEATS tagger:

--- Animacy:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Aspect:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Case:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Degree:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Foreign:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Gender:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Mood:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Number:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Person:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Polarity:
Loading dataset... done.
Creating model... done.
Loading state_dict... done.

--- Tense:
Loading dataset... done.
Creating model... done.
Loading state

In [37]:
junky.clear_tqdm()

In [38]:
_ = tagger.predict(corpus.dev, save_to='corpora/_dev_feats-sep.conllu')

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 98.27it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 98.65it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 99.29it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 98.45it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 98.78it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:07<00:00, 97.83it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 98.98it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:07<00:00, 97.96it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:07<00:00, 97.41it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 98.53it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:07<00:00, 97.38it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:07<00:00, 96.86it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:08<00:00, 96.06it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:08<00:00, 95.83it/s] 


In [39]:
_ = tagger.evaluate(corpus.dev)

Evaluating FEATS


Load corpus
[====> 4900                                                        

Processing corpus
100%|██████████| 6584/6584 [01:04<00:00, 102.25it/s]
Processing corpus
100%|██████████| 6584/6584 [01:04<00:00, 102.21it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 99.88it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.83it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.61it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.21it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.05it/s]
Processing corpus
100%|██████████| 6584/6584 [01:06<00:00, 99.68it/s] 
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.68it/s]
Processing corpus
100%|██████████| 6584/6584 [01:04<00:00, 101.46it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 101.23it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 101.17it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.47it/s]
Processing corpus
100%|██████████| 6584/6584 [01:05<00:00, 100.85it/s]
FEATS 

[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [40]:
_ = tagger.evaluate(corpus.dev, 'corpora/_dev_feats-sep.conllu')

Evaluating FEATS
Load corpus
[> 800                                                            

Load corpus
[> 800                                                            

[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
FEATS total: 72950 tokens, 287304 tags
    correct: 70490 tokens, 283962 tags
      wrong: 2460 tokens, 3342 tags [615 excess / 580 absent / 2147 wrong type]
   Accuracy: 0.9662782727895819 / 0.9883677219948208


[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [41]:
_ = tagger.evaluate(corpus.test)

Evaluating FEATS


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 102.81it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 102.69it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 102.21it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 101.82it/s]
Processing corpus
100%|██████████| 6491/6491 [01:04<00:00, 101.32it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 101.61it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 101.59it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 101.91it/s]
Processing corpus
100%|██████████| 6491/6491 [01:04<00:00, 101.30it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 102.32it/s]
Processing corpus
100%|██████████| 6491/6491 [01:04<00:00, 101.40it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 101.92it/s]
Processing corpus
100%|██████████| 6491/6491 [01:03<00:00, 102.49it/s]
Processing corpus
100%|██████████| 6491/6491 [01:04<00:00, 100.49it/s]
FEATS 

In [42]:
corp = list(syntagrus.dev())
corp_test = list(tagger._get_corpus('corpora/_dev_feats-sep.conllu'))
tags = sorted(set(x for x in corp for x in x[0] for x in x['FEATS'].keys()))
for tag in tags:
    print('{}: {}'.format(
        tag, tagger.evaluate(corp, corp_test,
                             feat=tag, log_file=None)
    ))

Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
Animacy: 0.98232644103038
Aspect: 0.9935810569859497
Case: 0.9803981506247373
Degree: 0.9956454477740216
Foreign: 0.9066985645933014
Gender: 0.9853467336683417
Mood: 0.9965310627562283
Number: 0.9928716238310117
Person: 0.9974799881411207
Polarity: 0.6792452830188679
Tense: 0.9950950864813699
Variant: 0.9736164736164736
VerbForm: 0.9987876194551419
Voice: 0.9891575718667522
